In [1]:
import os
import urllib.request
import pandas as pd
import csv
import re
import time
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 크롬 시작 && 네이버 오픈
def openNaver(driver):
    driver.get("https://www.naver.com/")

In [3]:
# 로그인 버튼 클릭
def clickLoginBt(driver):
    naver_login = '''//*[@id="account"]/a'''
    driver.find_element_by_xpath(naver_login).click()

In [4]:
# 네이버 로그인
def loginNaver(driver):
    user_id = 'wjdtnwjdtn55'
    user_password = 'zmfhffldgkwk%6'

    # time.sleep(1)

    # 유저 아이디 비번 복붙으로 넣어주기
    tag_id = driver.find_element_by_name('id') 
    tag_pw = driver.find_element_by_name('pw')


    # 아이디 입력
    tag_id.click()
    pyperclip.copy(user_id)
#     print(user_id)
    # 맥은 커맨드 윈도우는 컨트롤
    tag_id.send_keys(Keys.COMMAND, "v")


    # 비번 입력
    tag_pw.click()
    pyperclip.copy(user_password)
#     print(user_password)
    # 맥은 커맨드 윈도우는 컨트롤
    tag_pw.send_keys(Keys.COMMAND, "v")


    # 로그인 버튼 클릭 (아이디 비번 입력 후에)
    login_btn = driver.find_element_by_id('log.login') 
    login_btn.click()

    # 로그인 실패시 처리
    try:
        login_error = driver.find_element_by_css_selector('#err_common > div > p') 
        print('로그인 실패 > ', login_error.text) 
    except: 
        print('로그인 성공')

In [5]:
def gotoStartPositiopn(driver, url):
    # 중고나라 메인 페이지로 이동
    # notice_list_url_start = "https://cafe.naver.com/joonggonara"
    notice_list_url_start = url
#     notice_list_url_start = "https://cafe.naver.com/joonggonara/855706233"
    driver.get(notice_list_url_start)
    
    # 중고나라 불량 거래 게시판으로 이동
    # bad_trade_docs_list = '''//*[@id="menuLink57"]'''
    # driver.find_element_by_xpath(bad_trade_docs_list).click()
    
    # frame switch
    driver.switch_to.frame("cafe_main")
    
    # 가장 최근 게시글 클릭 (시작 지점)
    # top_of_the_docs = '''//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a'''
    # driver.find_element_by_xpath(top_of_the_docs).click()

## 목록을 타고 내려가면서 글을 바꿔줌

In [6]:
def travel(nums, driver):
    next_button_xpath_first = '''//*[@id="app"]/div/div/div[5]/div[1]/div/div[2]/div/ul/li[2]/div[1]/a'''
    next_button_xpath_second = '''//*[@id="app"]/div/div/div[5]/div[1]/div/div[2]/div/ul/li[3]/div[1]/a'''
    next_button_xpath_else = '''//*[@id="app"]/div/div/div[5]/div[1]/div/div[2]/div/ul/li[4]/div[1]/a'''

    
    # if nums == 0:
    #     url = next_button_xpath_first
    # elif nums == 1:
    #     url = next_button_xpath_second
    # else:
    #     url = next_button_xpath_else

    url = next_button_xpath_else
    
    try:
        driver.find_element_by_xpath(url).click()
        
    except:
        print("ERROR :: 다음 게시글로 이동할 수 없습니다.")

In [7]:
def getAll(driver):
    title = driver.find_element_by_class_name("title_text").text
    writer = driver.find_element_by_class_name("nickname").text
    date = driver.find_element_by_class_name("date").text
    view_count = driver.find_element_by_class_name("count").text
    nums_reply = driver.find_element_by_class_name("num").text
    
    try:
        contents = driver.find_element_by_class_name("se-module-text").text
    except:
        contents = "해당 글에는 텍스트가 존재하지 않습니다."

    list_all = [title, writer, date, view_count, nums_reply, contents]
    return list_all

In [8]:
def goToCsv(path, docs, colum):
    junggo = pd.DataFrame(docs)
    try:
        junggo.columns = colum
    except:
        print("Warning :: Check Your CSV")
        return
    
    if not os.path.exists(path):
        junggo.to_csv(path, index = False, mode = "w", encoding = "utf-8-sig")
    else:
        junggo.to_csv(path, index = False, mode = "a", encoding = "utf-8-sig", header = False)    

In [9]:
def getDriver():
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(3)
    return driver

In [12]:
def whatDoIDo(driver, path, colum, cnt, restart, length):
    doc = []
    if cnt == length:
        restart = True
    
    for i in range(1, 601):
        cnt += 1
        doc.clear()
        doc.append(getAll(driver))
        goToCsv(path = path, docs = doc, colum = colum)

        if i % 100 == 0:
            print(i)
        else:
            print("*", end = "")

        travel(i, driver)
    
    driver.find_element_by_xpath('''//*[@id="app"]/div/div/div[2]/div[1]/div[3]/a[2]''').click()
    url = pyperclip.paste()
#     send_keys(Keys.COMMAND, "v")
    print("Restart")
    driver.quit()
    return url

In [11]:
def main(path, colum, count):
    restart = False
    cnt = 0
    url = "https://cafe.naver.com/joonggonara/849324912"
    
    while restart != True:
        driver = getDriver()

        # 네이버 이동
        openNaver(driver)

        # 로그인 실행
        clickLoginBt(driver)
        loginNaver(driver)

        # 시작 지점으로 이동
        gotoStartPositiopn(driver, url)

        # 크롤링
        url = whatDoIDo(driver, path, colum, cnt, restart, count)

    

In [ ]:
path = "junggonara_bad_trade_board.csv"
# colum = ["TITLE", "WRITER", "DATE", "VIEW", "REPLY", "CONTENTS", "IMAGES"]
colum = ["TITLE", "WRITER", "DATE", "VIEW", "REPLY", "CONTENTS"]

main(path, colum, 30000)